# MIRA: Final Project 2023
Professor: Josep Quintana, Robert Martí

Topic: Image registration of chest CT volumes: 4DCT DIR-Lab Challenge

Taiabur


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from pathlib import Path


: 

In [ ]:

MAIN_PATH       = Path('../FinalProject/').resolve()
TRAIN_DATA_PATH = MAIN_PATH / 'dataset' / 'train'
OUTPUT_DATA_PATH = MAIN_PATH / 'dataset' / 'output'


In [ ]:

# Create an empty list to store data for DataFrame
data = []

for i, dir in enumerate(TRAIN_DATA_PATH.iterdir()):
    id = dir.stem
    
    inhale_image_name = f'{id}_iBHCT.img'
    exhale_image_name = f'{id}_eBHCT.img'

    inhale_landmark_name = f'{id}_300_iBH_xyz_r1.txt'
    exhale_landmark_name = f'{id}_300_eBH_xyz_r1.txt'

In [ ]:

def VoxelMorphNet(input_shape=(256, 256, 256, 1)):
    """
    VoxelMorph network architecture for 3D medical image registration.

    Args:
    input_shape (tuple): Shape of the input volume. Default is (256, 256, 256, 1).

    Returns:
    model (tf.keras.Model): A TensorFlow Keras model instance.
    """

    # Input layers for the fixed and moving images
    fixed_input = layers.Input(shape=input_shape, name='fixed_input')
    moving_input = layers.Input(shape=input_shape, name='moving_input')

    # Concatenate inputs
    x = layers.Concatenate(axis=-1)([fixed_input, moving_input])

    # Downsampling through Conv3D layers
    x = layers.Conv3D(16, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.Conv3D(32, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.Conv3D(64, kernel_size=3, activation='relu', padding='same')(x)

    # Upsampling and establishing the flow field
    x = layers.Conv3DTranspose(32, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.Conv3DTranspose(16, kernel_size=3, activation='relu', padding='same')(x)
    flow = layers.Conv3D(3, kernel_size=3, activation=None, padding='same', name='flow_field')(x)

    # Create the Keras model
    model = models.Model(inputs=[fixed_input, moving_input], outputs=flow)

    return model
